**Libs That Are Required**

In [ ]:
pip install --upgrade transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install timm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.7 MB/s eta 0:00:00


## **Define Model Path & Testing**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/gemma3n"

# Load locally
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto", local_files_only=True)

# Sample Prompt
prompt = "What is the capital of France?"

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=50)

# Decode
print(tokenizer.decode(output[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

What is the capital of France?

The capital of France is Paris.



## **Neccsessary Library For App**

In [ ]:
pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.4 MB/s eta 0:00:00


In [ ]:
pip install gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


### **Dashboard Code**

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import torchvision.transforms as transforms
from gtts import gTTS
import os


# Dummy image processing
def process_image(img_path):
    return "🧠 Image processed and analyzed (demo only)."

# Text-to-speech
def speak_response(response):
    tts = gTTS(response)
    audio_path = "response.mp3"
    tts.save(audio_path)
    return audio_path

# Main chat function
def gemma_response(category, user_input, chat_history):
    context = {
        "Healthcare": "You are a helpful medical assistant.",
        "Crop Care": "You are a plant and crop disease expert."
    }
    prompt = context[category] + "\n\n"
    for user_msg, bot_msg in chat_history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {user_input}\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()
    chat_history.append((user_input, response))
    audio_path = speak_response(response)
    return chat_history, chat_history, audio_path

with gr.Blocks(theme=gr.themes.Soft(primary_hue="green", secondary_hue="emerald")) as demo:
    gr.Markdown("""
    # 🩺🌾 **DoctorPulse**
    Welcome to your offline AI-powered assistant for **healthcare** and **crop diagnostics**.
    Powered by **Gemma 3n**.
    """)

    category = gr.State()
    chat_history = gr.State([])

    with gr.Row():
        healthcare_btn = gr.Button("🩺 Healthcare")
        cropcare_btn = gr.Button("🌾 Crop Care")

    selected_category_text = gr.Markdown("### Selected Category: _None_", visible=False)

    chatbot_area = gr.Column(visible=False)
    with chatbot_area:
        chat = gr.Chatbot(label="AI Assistant", type="tuples")
        txt = gr.Textbox(placeholder="Type your question here...", label="💬 Your Message")
        mic = gr.Audio( type="filepath", label="🎙️ Speak Instead", format ="wav")
        image_input = gr.Image(type="filepath", label="📷 Upload Image")
        image_result = gr.Textbox(label="Image Analysis Result")
        send_btn = gr.Button("📩 Send")
        new_chat_btn = gr.Button("🔄 Start New Chat")
        audio_output = gr.Audio(label="🔈 AI Response Voice")

    def show_chat(selected):
        return gr.update(visible=True), selected, [], gr.update(value=f"### Selected Category: _{selected}_", visible=True)

    healthcare_btn.click(fn=show_chat,
                         inputs=gr.State("Healthcare"),
                         outputs=[chatbot_area, category, chat_history, selected_category_text])

    cropcare_btn.click(fn=show_chat,
                       inputs=gr.State("Crop Care"),
                       outputs=[chatbot_area, category, chat_history, selected_category_text])

    send_btn.click(fn=gemma_response,
                   inputs=[category, txt, chat_history],
                   outputs=[chat, chat_history, audio_output])

    mic.change(lambda x: ("Voice input received. Please press send!", x), inputs=mic, outputs=[txt, mic])

    image_input.change(fn=process_image, inputs=image_input, outputs=image_result)

    new_chat_btn.click(fn=lambda: ([], [], None), inputs=None, outputs=[chat, chat_history, audio_output])

if __name__ == "__main__":
    demo.launch()


/tmp/ipython-input-1548391872.py:58: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chat = gr.Chatbot(label="AI Assistant", type="tuples")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a7b757b6840c0de10.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
